In [6]:
# This notebooks downloads L1 products from the Finnish Data Hub System. 
# Inputs: start_date, end_data, area_of_interest

In [ ]:
import os

In [ ]:
import os
import shutil
import zipfile
from datetime import date
from pathlib import Path
from datetime import date, timedelta

from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt, make_path_filter

In [ ]:
S3_ROOT_FOLDER = Path('/home/jovyan/earth-observations/notebooks')
ROI_FOLDER = S3_ROOT_FOLDER / 'ROI'  

footprint = geojson_to_wkt(read_geojson(ROI_FOLDER / 'indre-oslofjord.geojson'))

In [3]:
# read env variables
api = SentinelAPI(os.getenv("FIN_HUB_USER"),
                  os.getenv("FIN_HUB_PASSWORD"),
                  "https://finhub.nsdc.fmi.fi/")

In [4]:
def query_products(footprint, start_date, end_date, maxclouds=100):
    
    # path_filter = make_path_filter("_NT_")

    query_parameters = {'timeliness': 'Non-Time Critical'}

    products = api.query(footprint,
                         date=(start_date, end_date),
                         # platformname='Sentinel-2', 
                         # cloudcoverpercentage=(0, 100),
                         producttype='OL_1_EFR___', 
                         **query_parameters)

    return products.keys()

In [5]:
def move_downloaded_files(destination_folder, roi, delete_zip=True):
        
    for zip_file in S3_ROOT_FOLDER.glob('*.zip'):
        try:
            
            # Create the destination folder if it doesn't exist
            destination_folder.mkdir(parents=True, exist_ok=True)

            # Move the ZIP file to the destination folder
            moved_zip_path = destination_folder / zip_file.name
            shutil.move(str(zip_file), moved_zip_path)
            
            # print(f"Moved {zip_file.name} to {destination_folder}")
            # Extract the ZIP file in the destination folder
            with zipfile.ZipFile(moved_zip_path, 'r') as zip_ref:
                zip_ref.extractall(destination_folder)
                print(f"Extracted {zip_file.name} to {destination_folder}")

                
            if delete_zip:
                # Delete the ZIP file after extraction
                moved_zip_path.unlink()
                # print(f"Deleted {moved_zip_path.name} after extraction")
                
        except ValueError as e:
            print(f"Skipping {zip_file.name}: {e}")

In [6]:
def download_S3_data(start_date, end_date, roi):
    # There must be a file in the ROI folder, called as the passed roi
    
    zip_files = [file for file in os.listdir('.') if file.endswith('.zip')]
    # Assert that there are no .zip files
    assert not zip_files, "Make sure there are no unwanted zip files in the notebook directory"
     
    delta = end_date - start_date
    
    footprint = geojson_to_wkt(read_geojson(ROI_FOLDER / (roi + '.geojson')))
    
    # Iterate over each day in the range
    for i in range(delta.days + 1):
        
        current_date = start_date + timedelta(days=i)
        
        # Extract metadata from the filename
        year, month, day =  str(current_date.year), current_date.strftime('%B'), str(current_date.day)
        
        # Construct the destination folder path
        destination_folder = S3_ROOT_FOLDER / 'S3' / 'L1' / roi / year / month / day
        
        # Create the destination folder if it doesn't exist
        if destination_folder.exists() and any(destination_folder.iterdir()):
            print(f"Skipping download. Destination folder {destination_folder} already exists and is not empty")
            continue
        
        products = query_products(footprint, current_date, current_date + timedelta(days=1))
        
        print(current_date, ": " + str(len(products)) + " products")
        
        api.download_all(products)
        
        move_downloaded_files(destination_folder, roi)
    
    print("Done!")

In [8]:
download_S3_data(date(2024, 3, 18), date(2024, 3, 21), "indre-oslofjord")

2024-03-18 : 2 products


MD5 checksumming:   0%|          | 0.00/892M [00:00<?, ?B/s]

MD5 checksumming:   0%|          | 0.00/900M [00:00<?, ?B/s]

Extracted S3A_OL_1_EFR____20240318T103235_20240318T103535_20240319T105723_0179_110_165_1980_PS1_O_NT_003.SAFE.zip to /home/jovyan/earth-observations/notebooks/S3/L1/indre-oslofjord/2024/March/18
Extracted S3B_OL_1_EFR____20240318T095336_20240318T095636_20240318T225446_0179_091_022_1980_PS2_O_NT_003.SAFE.zip to /home/jovyan/earth-observations/notebooks/S3/L1/indre-oslofjord/2024/March/18
2024-03-19 : 2 products


MD5 checksumming:   0%|          | 0.00/907M [00:00<?, ?B/s]

...

MD5 checksumming:   0%|          | 0.00/838M [00:00<?, ?B/s]

Extracted S3A_OL_1_EFR____20240320T094014_20240320T094314_20240321T100317_0180_110_193_1980_PS1_O_NT_003.SAFE.zip to /home/jovyan/earth-observations/notebooks/S3/L1/indre-oslofjord/2024/March/20
Extracted S3B_OL_1_EFR____20240320T104213_20240320T104513_20240320T230658_0180_091_051_1980_PS2_O_NT_003.SAFE.zip to /home/jovyan/earth-observations/notebooks/S3/L1/indre-oslofjord/2024/March/20
2024-03-21 : 2 products


MD5 checksumming:   0%|          | 0.00/892M [00:00<?, ?B/s]

MD5 checksumming:   0%|          | 0.00/847M [00:00<?, ?B/s]

Extracted S3A_OL_1_EFR____20240321T091403_20240321T091703_20240322T093702_0179_110_207_1980_PS1_O_NT_003.SAFE.zip to /home/jovyan/earth-observations/notebooks/S3/L1/indre-oslofjord/2024/March/21
Extracted S3B_OL_1_EFR____20240321T101602_20240321T101902_20240321T225407_0179_091_065_1980_PS2_O_NT_003.SAFE.zip to /home/jovyan/earth-observations/notebooks/S3/L1/indre-oslofjord/2024/March/21
Done!


In [76]:
# query_products(footprint, start_date, end_date, producttype)
# api.download_all_quicklooks(products)
# api.download_all(products)